In [1]:
!pip install pyarrow

In [26]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [3]:
data_jan=pd.read_parquet("./dataset/yellow_tripdata_2023-01.parquet")

### Part 1

In [4]:
data_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

### Part 2

In [5]:
data_jan['duration']= data_jan['tpep_dropoff_datetime']-data_jan['tpep_pickup_datetime']

In [6]:
data_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,0 days 00:08:26
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,0 days 00:06:19
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,0 days 00:12:45
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,0 days 00:09:37
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,0 days 00:10:50


In [7]:
data_jan['duration'].std()

Timedelta('0 days 00:42:35.661074')

### Part 3



In [8]:
orig_count = data_jan['duration'].describe()['count']

In [9]:
filtered_data_jan=data_jan[(data_jan['duration']>=pd.Timedelta(minutes=1)) & (data_jan['duration']<=pd.Timedelta(minutes=60))]

In [10]:
filtered_count=filtered_data_jan['duration'].describe()['count']

In [11]:
fraction_left=filtered_count/orig_count

In [12]:
print(fraction_left)

0.9812202822125979


In [13]:
filtered_data_jan['duration'] = filtered_data_jan['duration'].dt.total_seconds()

C:\Users\skhatta\AppData\Local\Temp\ipykernel_26516\3710301738.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data_jan['duration'] = filtered_data_jan['duration'].dt.total_seconds()


In [14]:
filtered_data_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,506.0
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,379.0
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,765.0
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,577.0
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,650.0


### Part 4

In [15]:
data_train = filtered_data_jan[['PULocationID','DOLocationID','duration']]

In [16]:
data_train['PULocationID'] = data_train['PULocationID'].astype(str)
data_train['DOLocationID'] = data_train['DOLocationID'].astype(str)

C:\Users\skhatta\AppData\Local\Temp\ipykernel_26516\1300438918.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['PULocationID'] = data_train['PULocationID'].astype(str)
C:\Users\skhatta\AppData\Local\Temp\ipykernel_26516\1300438918.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['DOLocationID'] = data_train['DOLocationID'].astype(str)


In [17]:
len(data_train)

3009173

In [18]:
# Convert DataFrame to a list of dictionaries
data_list = data_train.to_dict(orient='records')

In [19]:
# Fit a DictVectorizer
vectorizer = DictVectorizer(sparse=False)
feature_matrix = vectorizer.fit_transform(data_list)

In [20]:
# Get feature names from DictVectorizer
feature_names = vectorizer.get_feature_names_out()

# Convert feature matrix to DataFrame for visualization (optional)
feature_df = pd.DataFrame(feature_matrix, columns=feature_names)

print(feature_df.shape)

(3009173, 516)


In [21]:
X=feature_df.iloc[:,:-1]
y=feature_df.iloc[:,-1]    

### Part 5

In [22]:
model=LinearRegression()
model.fit(X,y)

LinearRegression()

In [23]:
y_pred=model.predict(X)

In [27]:
rmse=np.sqrt(mean_squared_error(y, y_pred))

In [29]:
print(rmse/60)

7.6492617359215025


### Part 6

In [30]:
def prepare_data(path):
    data= pd.read_parquet(path)
    data['duration']= data['tpep_dropoff_datetime']-data['tpep_pickup_datetime']
    filtered_data=data[(data['duration']>=pd.Timedelta(minutes=1)) & (data['duration']<=pd.Timedelta(minutes=60))]
    filtered_data['duration'] = filtered_data['duration'].dt.total_seconds()
    data_sel = filtered_data[['PULocationID','DOLocationID','duration']]
    data_sel['PULocationID'] = data_sel['PULocationID'].astype(str)
    data_sel['DOLocationID'] = data_sel['DOLocationID'].astype(str)
    # Convert DataFrame to a list of dictionaries
    data_list = data_sel.to_dict(orient='records')
    # Fit a DictVectorizer
    vectorizer = DictVectorizer(sparse=False)
    feature_matrix = vectorizer.fit_transform(data_list)
    # Get feature names from DictVectorizer
    feature_names = vectorizer.get_feature_names_out()

    # Convert feature matrix to DataFrame for visualization (optional)
    feature_df = pd.DataFrame(feature_matrix, columns=feature_names)
    X=feature_df.iloc[:,:-1]
    y=feature_df.iloc[:,-1]

    return X,y
    

In [31]:
X_val,y_val=prepare_data('./dataset/yellow_tripdata_2023-02.parquet')

C:\Users\skhatta\AppData\Local\Temp\ipykernel_26516\1182883919.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['duration'] = filtered_data['duration'].dt.total_seconds()
C:\Users\skhatta\AppData\Local\Temp\ipykernel_26516\1182883919.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sel['PULocationID'] = data_sel['PULocationID'].astype(str)
C:\Users\skhatta\AppData\Local\Temp\ipykernel_26516\1182883919.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [35]:
print(X_val.shape)

(2855951, 514)


In [33]:
y_val_pred=model.predict(X_val)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- DOLocationID=105
- PULocationID=105
- PULocationID=204
- PULocationID=59
Feature names seen at fit time, yet now missing:
- PULocationID=111
- PULocationID=187
- PULocationID=199
- PULocationID=206
- PULocationID=251
